In [1]:
upstream = ['01-get']
product = None

import re

In [2]:
# Parameters
upstream = {"01-get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/data/house_prices.csv"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/cleanup.ipynb", "clean_csv": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/data/house_prices_clean.csv"}


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(upstream['01-get']['csv_file'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO.                29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


In [6]:
df.isna().sum()

POSTED_BY                0
UNDER_CONSTRUCTION       0
RERA                     0
BHK_NO.                  0
BHK_OR_RK                0
SQUARE_FT                0
READY_TO_MOVE            0
RESALE                   0
ADDRESS                  0
LONGITUDE                0
LATITUDE                 0
TARGET(PRICE_IN_LACS)    0
dtype: int64

In [7]:
# Rename columns to snake case
renamed_mapping = {}
for c in df.columns:
    new_c_name = re.sub('[^0-9a-zA-Z_]+', '', c)
    renamed_mapping[c] = new_c_name.lower()  # .replace(' ', '_').replace('(', '').replace(')', '')
renamed_mapping['targetprice_in_lacs'] = 'target_price_in_lacs'

df = df.rename(columns=renamed_mapping)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   posted_by            29451 non-null  object 
 1   under_construction   29451 non-null  int64  
 2   rera                 29451 non-null  int64  
 3   bhk_no               29451 non-null  int64  
 4   bhk_or_rk            29451 non-null  object 
 5   square_ft            29451 non-null  float64
 6   ready_to_move        29451 non-null  int64  
 7   resale               29451 non-null  int64  
 8   address              29451 non-null  object 
 9   longitude            29451 non-null  float64
 10  latitude             29451 non-null  float64
 11  targetprice_in_lacs  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


In [8]:
# Extracting cities from address
def get_city(value):
    parts = value.split(',')
    return parts[-1:][0]

df['city'] = df['address'].apply(get_city)

df['city'].value_counts()

city
Bangalore      4340
Lalitpur       2993
Mumbai         2023
Pune           1991
Noida          1767
               ... 
Bhadrak           1
Kurukshetra       1
Dibrugarh         1
Sagar             1
Washim            1
Name: count, Length: 256, dtype: int64

In [9]:
df = df.drop(columns=['address'])

In [10]:
df['posted_by'].value_counts()

posted_by
Dealer     18291
Owner      10538
Builder      622
Name: count, dtype: int64

In [11]:
df['bhk_or_rk'].value_counts()

bhk_or_rk
BHK    29427
RK        24
Name: count, dtype: int64

In [12]:
df = pd.get_dummies(df, columns=['posted_by', 'bhk_or_rk', 'city'])

In [13]:
df.head()

,under_construction,rera,bhk_no,square_ft,ready_to_move,resale,longitude,latitude,targetprice_in_lacs,posted_by_Builder,...,city_Vellore,city_Vidisha,city_Vijayawada,city_Visakhapatnam,city_Visnagar,city_Vizianagaram,city_Warangal,city_Wardha,city_Washim,city_Yavatmal
0,0,0,2,1300.236407,1,1,12.969910,77.597960,55.0,False,...,False,False,False,False,False,False,False,False,False,False
1,0,0,2,1275.000000,1,1,12.274538,76.644605,51.0,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0,2,933.159722,1,1,12.778033,77.632191,43.0,False,...,False,False,False,False,False,False,False,False,False,False
3,0,1,2,929.921143,1,1,28.642300,77.344500,62.5,False,...,False,False,False,False,False,False,False,False,False,False
4,1,0,2,999.009247,0,1,22.592200,88.484911,60.5,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Columns: 270 entries, under_construction to city_Yavatmal
dtypes: bool(261), float64(4), int64(5)
memory usage: 9.4 MB


In [15]:
for c in df.columns:
    if df[c].dtype == bool:
        print(f'{c} {df[c].dtype}')
        df[c] = df[c].astype(int)

df.info()

posted_by_Builder bool
posted_by_Dealer bool
posted_by_Owner bool
bhk_or_rk_BHK bool
bhk_or_rk_RK bool
city_Agartala bool
city_Agra bool
city_Ahmednagar bool
city_Ajmer bool
city_Akola bool
city_Aligarh bool
city_Allahabad bool
city_Alwar bool
city_Ambala bool
city_Amravati bool
city_Amritsar bool
city_Amroha bool
city_Anand bool
city_Anantapur bool
city_Angul bool
city_Arrah bool
city_Asansol bool
city_Aurangabad bool
city_Baddi bool
city_Bahadurgarh bool
city_Balasore bool
city_Bangalore bool
city_Bankura bool
city_Banswara bool
city_Barabanki bool
city_Bardhaman bool
city_Bareilly bool
city_Beed bool
city_Belgaum bool
city_Berhampur bool
city_Betul bool
city_Bhadrak bool
city_Bhagalpur bool
city_Bhandara bool
city_Bharatpur bool
city_Bharuch bool
city_Bhavnagar bool
city_Bhilai bool
city_Bhimavaram bool
city_Bhiwadi bool
city_Bhiwani bool
city_Bhopal bool
city_Bhubaneswar bool
city_Bhuj bool
city_Bikaner bool
city_Bilaspur bool
city_Bokaro bool
city_Chandigarh bool
city_Chandrapur b

city_Nagapattinam bool
city_Nagaur bool
city_Nagpur bool
city_Nainital bool
city_Nanded bool
city_Navsari bool
city_Neemrana bool
city_Nellore bool
city_Nizamabad bool
city_Noida bool
city_Ongole bool
city_Ooty bool
city_Osmanabad bool
city_Ottapalam bool
city_Palakkad bool
city_Palanpur bool
city_Palghar bool
city_Pali bool
city_Palwal bool
city_Panaji bool
city_Panchkula bool
city_Panipat bool
city_Pathanamthitta bool
city_Patiala bool
city_Patna bool
city_Phagwara bool
city_Pondicherry bool
city_Porbandar bool
city_Pudukkottai bool
city_Pune bool
city_Puri bool
city_Raichur bool
city_Raigad bool
city_Raigarh bool
city_Raipur bool
city_Raisen bool
city_Rajkot bool
city_Rajnandgaon bool
city_Rajpura bool
city_Ranchi bool
city_Ratnagiri bool
city_Rayagada bool
city_Rewa bool
city_Rewari bool
city_Rishikesh bool
city_Rohtak bool
city_Roorkee bool
city_Rourkela bool
city_Rudrapur bool
city_Sabarkantha bool
city_Sagar bool
city_Salem bool
city_Sambalpur bool
city_Satna bool
city_Secundera

In [16]:
# 1. hot shoe encoding
# 2. Drop encoded columns
# 3. Extract city
# 4.

In [17]:

df.to_csv(product['clean_csv'], index=False)